In [69]:
import requests as requests
import re
import pandas as pd

In [93]:
def get_pageview_data(articles, years, lang, save_csv=True):
    search_url = f'http://petermeissner.de:8880/article/exact/{lang}/'
    final_list = []
    
    for article in articles:
        article_no_spaces = re.sub(r'\s','_', article)
        request = requests.get(search_url + article_no_spaces).json()
        if request['status'] != 'ok':
            print(f'Problem fetching data for artice {article}')
            continue
        data = request['data']
        for elem in data:
            year = elem['year']
            if str(year) in years:
                timestamps = pd.date_range(f'01-01-{year}', f'31-12-{year}')
                page_views = elem['page_view_count'].split(',')
                assert len(timestamps) == len(page_views)
                final_list += list(zip(timestamps, [article] * len(timestamps), [lang] * len(timestamps), page_views)) 
    df = pd.DataFrame(final_list, columns=['date', 'article','language', 'pageviews'])
    if save_csv:
        df.to_csv(f'data/data_{lang}.csv')
    return df
    

In [145]:
terrorism_articles = ['abu_sayyaf', 'afghanistan', 'agro', 'al-qaeda',
       'al-qaeda_in_the_arabian_peninsula',
       'al-qaeda_in_the_islamic_maghreb', 'al-shabaab',
       'ammonium_nitrate', 'attack', 'biological_weapon', 'car_bomb',
       'chemical_weapon', 'conventional_weapon', 'dirty_bomb',
       'eco-terrorism', 'environmental_terrorism',
       'euskadi_ta_askatasuna', 'extremism', 'farc', 'fundamentalism',
       'hamas', 'hezbollah', 'improvised_explosive_device', 'iran',
       'iraq', 'irish_republican_army', 'islamist', 'jihad',
       'nationalism', 'nigeria', 'nuclear', 'nuclear_enrichment',
       'pakistan', 'palestine_liberation_front', 'pirates', 'plo',
       'political_radicalism', 'recruitment', 'somalia', 'suicide_attack',
       'suicide_bomber', 'taliban', 'tamil_tigers',
       'tehrik-i-taliban_pakistan', 'terror', 'terrorism',
       'weapons-grade', 'yemen']

In [146]:
articles = get_pageview_data(terrorism_articles, ['2012', '2013','2014','2015', '2016'], 'en')

In [161]:
reference = pd.read_csv('data/terrorism_data.csv')

In [191]:
testing_ref_data = reference.set_index(['date','article']).loc[:, 'views'].astype(int).sort_index(level=[0,1])

In [218]:
test = articles.loc[articles.date.isin(reference.date)].set_index(['date','article']).sort_index(level=[0,1])['pageviews'].astype(int)

In [219]:
import numpy as np

np.count_nonzero((test.values == testing_ref_data.values))

46752

In [220]:
test.shape

(46752,)